In [1]:
import geopandas as gpd
import fiona
import osmnx as ox
import folium
from fiona.model import to_dict
from shapely.geometry import shape
from branca.colormap import LinearColormap
import numpy as np
import networkx as nx
import pandas as pd

import matplotlib.pyplot as plt
from scipy.spatial import Voronoi,voronoi_plot_2d
from geovoronoi import voronoi_regions_from_coords, points_to_coords
import random
from shapely.geometry import Polygon, Point

from skmob.tessellation.tilers import tiler
from skmob.utils.plot import plot_gdf
import requests
import pyproj
from shapely.ops import transform

In [2]:
def Leuven_map():
    point = (50.849738563227824, 4.644786576420597)
    m = folium.Map(location=point, zoom_start = 12)
    return m

# DATA

In [3]:
zones_leuven = gpd.read_file('https://storageaccount11111111.blob.core.windows.net/container1/Leuven/socio_demographic_data/leuven_statsec.gpkg')
zones_leuven = zones_leuven.to_crs('epsg:4326')
zones_leuven.reset_index(inplace=True)

In [4]:
# ciclabile/strada
cyclic = gpd.read_file('https://storageaccount11111111.blob.core.windows.net/container1/Leuven/cycling_data/cycling_network.gpkg')
cyclic = cyclic.to_crs(epsg=4326)

In [5]:
#info socio-dem
socio_dem_data = pd.read_excel('https://storageaccount11111111.blob.core.windows.net/container1/Leuven/socio_demographic_data/socio_demographic_data_leuven_2019.xlsx')
#drop nan zone
socio_dem_data.drop(np.where(socio_dem_data['CODSEC'].isna())[0][0], axis = 0 ,inplace=True)

In [6]:
#ciclabili per le zone selezionate
ind = set()
for i in range(len(cyclic)):
    z = zones_leuven['geometry']
    s = np.where(z.intersects(cyclic['geometry'][i]))[0]
    if len(s) > 0:
        ind.add(i)

In [7]:
leuven_cyclic = cyclic.iloc[list(ind), :]
utm = leuven_cyclic.estimate_utm_crs()
project = pyproj.Transformer.from_crs('4326', utm)

In [8]:
# off_street_parking
url = 'https://storageaccount11111111.blob.core.windows.net/container1/Leuven/parking_data_leuven/off_street_parking_supply_Leuven.json'
response = requests.get(url)
json_data = response.json()

results = json_data['JSON']['results']
normalized_data = pd.json_normalize(results)

df = pd.DataFrame(normalized_data)

In [9]:
# estraggo info dalla colonna bullets 
spots = []
cameras = []
accessibility = []
lift = []
height = []


for row in df['bullets']:
    spot_number = None
    camera_label = None
    accessibility_label = None
    lift_label = None
    height_label = None
    
    for item in row:
        label = item['label']
        if 'spots' in item['icon']:
            spot_number = int(''.join(filter(str.isdigit, label)))
        elif 'camera' in item['icon']:
            camera_label = label
        elif 'accessibility' in item['icon']:
            accessibility_label = label
        elif 'lift' in item['icon']:
            lift_label = label
        elif 'height' in item['icon']:
            height_label = label
    

    spots.append(spot_number)
    cameras.append(camera_label)
    accessibility.append(accessibility_label)
    lift.append(lift_label)
    height.append(height_label)

df['spots'] = spots
df['cameras'] = cameras
df['accessibility'] = accessibility
df['lift'] = lift
df['height'] = height

df = df.drop('bullets', axis=1)

In [10]:
geometry = [Point(xy) for xy in zip(df['coords.lng'], df['coords.lat'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

columns_subset = ['pricing_type', 'free_bus', 'id', 'subtype', 'spots', 'lift', 'geometry']
gdf = gdf[columns_subset]
mapping = {'None': 0, 'Parking met lift': 1, True: 1, False:0}
gdf['lift'] = gdf['lift'].replace(mapping)
gdf['free_bus'] = gdf['free_bus'].replace(mapping)
gdf['lift'] = gdf['lift'].fillna(0)
gdf.shape

(43, 7)

In [11]:
stops = pd.read_csv('stops.csv')

In [12]:
coords = []
for index, rows in stops.iterrows():
    coords.append(Point(rows['stop_lon'], rows['stop_lat'],))

coords = gpd.GeoDataFrame(geometry=coords, crs = 'EPSG:4326')
coords['id'] = stops['stop_id']

In [13]:
#stops per le zone selezionate
ind = set()
for i in range(len(coords)):
    z = zones_leuven['geometry']
    s = np.where(z.contains(coords['geometry'][i]))[0]
    if len(s) > 0:
        ind.add(i)

In [14]:
leuven_stops = coords.iloc[list(ind), :]
leuven_stops.reset_index(inplace=True)

In [15]:
import geopy.distance

In [16]:
utm_coords = leuven_stops.to_crs(utm)

In [17]:
utm_coords.reset_index(inplace = True)

In [18]:
m = Leuven_map()
for ind, row in leuven_stops.iterrows():
    folium.Marker(row['geometry'].coords[0][::-1], popup=f"<id>{row['id']}").add_to(m)

In [19]:
ind = set()
for i,v in utm_coords.iterrows():
    if i not in ind:
        dist_arr = np.array(v['geometry'].distance(utm_coords['geometry']))
        low_arr = set(np.where(dist_arr < 60)[0])
        zero_dist = set(np.where(dist_arr == 0)[0])
        if len(zero_dist) > 0:
            low_arr = low_arr.difference(zero_dist)
        ind.update(low_arr)

In [20]:
ridotto = leuven_stops.drop(ind, axis = 0)
ridotto.reset_index(inplace=True)
ridotto_utm = ridotto.to_crs(utm)

## calcolo distanze tra :

In [82]:
transformed_parkings = gdf.to_crs(utm)
transform_cyclic = leuven_cyclic.to_crs(utm)

candidates = []
for candidate in transform_cyclic['geometry']:
    feat = {}
    #distance from off-street parking
    feat['nearest_park'] = np.min(candidate.distance(transformed_parkings['geometry']))
    #distance from center by bike
    #bus stops
    feat['nearest_bus'] = np.min(candidate.distance(ridotto_utm['geometry']))
    #find the city zone of the candidate street
    zona = np.where(candidate.intersects(zones_leuven['geometry'].to_crs(utm)))[0][0]
    #retrieve demographic information
    code = zones_leuven['CODSEC'][zona]
    feat['total_citizens'] = float(socio_dem_data[socio_dem_data['CODSEC'] == code]['total_citizens'])
    feat['outg_commuters'] =float(socio_dem_data[socio_dem_data['CODSEC'] == code]['outg_commuters'])
    feat['inc_commuters'] =float(socio_dem_data[socio_dem_data['CODSEC'] == code]['inc_commuters'])
    candidates.append(feat)
    

In [83]:
candidates

[{'nearest_park': 2718.9982369141817,
  'nearest_bus': 576.8247760931669,
  'total_citizens': 155.0,
  'outg_commuters': 29.0,
  'inc_commuters': 1959.1600565142799},
 {'nearest_park': 2880.568954024304,
  'nearest_bus': 1549.3314180716884,
  'total_citizens': 18.0,
  'outg_commuters': 6.0,
  'inc_commuters': 0.0},
 {'nearest_park': 2243.4325296431925,
  'nearest_bus': 561.8361074515535,
  'total_citizens': 102.0,
  'outg_commuters': 23.0,
  'inc_commuters': 214.75069801863626},
 {'nearest_park': 1831.9803964732985,
  'nearest_bus': 130.3080825952825,
  'total_citizens': 102.0,
  'outg_commuters': 23.0,
  'inc_commuters': 214.75069801863626},
 {'nearest_park': 1718.185444388787,
  'nearest_bus': 152.78724411714066,
  'total_citizens': 179.0,
  'outg_commuters': 29.0,
  'inc_commuters': 80.9715746627645},
 {'nearest_park': 1826.176104601739,
  'nearest_bus': 224.71602535533862,
  'total_citizens': 179.0,
  'outg_commuters': 29.0,
  'inc_commuters': 80.9715746627645},
 {'nearest_park': 2